# Harvest records from zbMath
This notebook does basically the same as the [Access data from zbMath](access_data_zbMath.ipynb), but uses the ListRecords endpoint to harvest data *en masse*. This is both more efficient as less http requests are sent, and easier to programm.

Data in [zbMath Open](https://www.zbmath.org/) can be accessed through the [zbMath Open OAI-PMH](https://oai.zbmath.org/) service, that implements the Open Archives Initiative Protocol for Metadata Harvesting (OAI-PMH) [Schubotz and Teschke, 2021]. The service is open and subject to certain [terms and conditions](https://oai.zbmath.org/static/terms-and-conditions.html).

In [1]:
%run zbMath_common.ipynb

## Get a list of records

In [2]:
import requests

date_from = '2021-01-01T00%3A00%3A00Z'
date_until = '2022-01-01T00%3A00%3A00Z'
REQUEST_URL="{}&from={}&until={}".format(LIST_RECORDS, date_from, date_until)

# get data from API
headers = {'accept': 'text/xml'} # this has NO effect
all_records_xml = requests.get(REQUEST_URL, headers)

# save raw data in local file
with open('data/all_records.xml', 'w') as f:
    f.write(all_records_xml.text)

Parse the response into an XML tree, and find all the 'record' XML elements

In [3]:
import xml.etree.ElementTree as ET
import pandas as pd

#parse the tree, get a list of records
tree = ET.parse('data/all_records.xml')
list_ids = tree.getroot().find(ns('ListRecords'))
records = list_ids.findall(ns('record'))

Parse all records in the data set, put them into a pandas data frame

In [4]:
# loop through all entries
all_details = []
for record in records:
    details = parse_record(record)
    if details:
        all_details.append(details)

# convert to data frame
records_df = pd.DataFrame(all_details)
if 'id' not in records_df.columns:
    print("Problem reading zbMath id's. No data?")
else:
    records_df.set_index('id', inplace=True)

print('Imported {} entries (discarded {} for licensing conflicts)'.format(len(records_df), len(records) - len(records_df) ))

Imported 46 entries (discarded 54 for licensing conflicts)


Save the data for later usage

In [5]:
#records_df.to_csv('data/records.csv')